In [5]:
import numpy as np
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
from sklearn import metrics
from sklearn import datasets
import matplotlib.pyplot as plt
from xgboost import plot_importance
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier as GBC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, auc

In [6]:
data = pd.read_csv('train.csv')
quiz = pd.read_csv('test_data.csv')

In [7]:

drop_out = [data.columns[0]]
for i in data.columns[5:]:
    if 1-len(data[data[i].notna()])/len(data)>0.3:
        drop_out.append(i)
print(drop_out)
data = data.drop(columns=drop_out)
quiz = quiz.drop(columns=drop_out)
data

['Unnamed: 0', 'COL13', 'COL14', 'COL21', 'COL25', 'COL31', 'COL37', 'COL45', 'COL46', 'COL52']


ID  CHANNEL_A  CHANNEL_B  CHANNEL_C   COL1  COL2 COL3 COL4 COL5  \
0      T000689          0          0          0    NaN    58    A    B    E   
1      T000848          0          1          0    6.0    49    B    B    C   
2      T000967          0          0          0    NaN    55    B    A    C   
3      T001249          0          0          0   17.0    36    K    A    B   
4      T001524          0          1          0   22.0    43  NaN  NaN    B   
...        ...        ...        ...        ...    ...   ...  ...  ...  ...   
99995  T034492          0          1          0    NaN    30    K    A    A   
99996  T034682          0          0          0    4.0    49    Z    B    B   
99997  T035207          0          0          0   18.0    30    B    A    A   
99998  T035311          0          0          0  347.0    33    B    B    A   
99999  T037255          0          1          0    NaN    34    B    B    B   

        COL6  ...  COL42  COL43  COL44  COL47  COL48  COL49  COL50  COL51  \
0       0.00  ...   0.00   3.09    0.0   0.00    0.0    2.0  11.23  318.0   
1       0.00  ... -37.56  59.94    0.0   0.00    0.0    4.0  48.20    NaN   
2       0.00  ...  24.66  10.00    0.0   0.00    1.0    1.0  36.84   49.0   
3       0.00  ...   0.00  24.16    0.0  23.82    0.0    0.0   0.00   49.0   
4       0.00  ...   0.00  26.30    0.0   0.00    0.0    0.0   0.00   17.0   
...      ...  ...    ...    ...    ...    ...    ...    ...    ...    ...   
99995   0.00  ...   0.00   0.00    0.0   0.00    0.0    0.0   0.00    NaN   
99996   0.00  ...   0.00  20.88    1.0   0.00    1.0    0.0   0.00    4.0   
99997  60.78  ...   0.00  20.29    1.0  24.80    1.0    0.0   0.00   17.0   
99998   9.74  ...   0.00  28.36    1.0   0.00    1.0    0.0   0.00  292.0   
99999  58.43  ...  14.42  11.67    1.0  40.96    1.0    0.0   0.00   12.0   

       COL53  COL54  
0        0.0    0.0  
1        0.0    0.0  
2        0.0    0.0  
3        1.0    0.0  
4        0.0    0.0  
...      ...    ...  
99995    0.0    0.0  
99996    0.0    0.0  
99997    0.0    3.0  
99998    0.0    0.0  
99999    0.0    0.0  

[100000 rows x 49 columns]

In [8]:
quiz

ID   COL1  COL2 COL3 COL4 COL5   COL6  COL7  COL8   COL9  ...  \
0      T500001   26.0  54.0    B    B    C   0.00   0.0   1.0   63.0  ...   
1      T500002   22.0  55.0    C    B    B   0.92   0.0   1.0   43.0  ...   
2      T500003    NaN  55.0    B    B    B   0.00   0.0   0.0    NaN  ...   
3      T500004  106.0  47.0    C    B    C  22.46   0.0   1.0  185.0  ...   
4      T500005   18.0  55.0    C    B    C  22.26   2.0   0.0   43.0  ...   
...        ...    ...   ...  ...  ...  ...    ...   ...   ...    ...  ...   
49996  T549996   39.0  30.0    B    B    A  55.75   0.0   1.0   72.0  ...   
49997  T549997   15.0  37.0    A    B    C   0.00   0.0   0.0   38.0  ...   
49998  T549998   46.0  29.0    Z    A    A   0.46   0.0   0.0   65.0  ...   
49999  T549999   61.0  35.0    B    A    B   0.00   0.0   1.0   98.0  ...   
50000  T550000   36.0  41.0    B    B    C   0.00   0.0   0.0   58.0  ...   

       COL42  COL43  COL44  COL47  COL48  COL49  COL50  COL51  COL53  COL54  
0       0.00  25.58    1.0  57.08    1.0    0.0   0.00   64.0    0.0    0.0  
1      21.94  14.74    1.0   0.00    1.0    0.0   0.00  240.0    0.0    0.0  
2     -21.97   0.00    0.0  37.38    0.0    0.0   0.00    NaN    0.0    0.0  
3       0.00   0.00    0.0   0.00    0.0    0.0   0.00  164.0    0.0    0.0  
4       0.00   0.00    1.0   0.00    1.0    0.0   0.00   35.0    0.0    0.0  
...      ...    ...    ...    ...    ...    ...    ...    ...    ...    ...  
49996  66.31   0.00    0.0   0.00    1.0    0.0   0.00    NaN    1.0    0.0  
49997   0.00  11.87    0.0  33.44    0.0    2.0  41.21    NaN    1.0    0.0  
49998   0.00  18.93    1.0  17.37    1.0    0.0   0.00    NaN    1.0    0.0  
49999   0.00  27.03    0.0   0.00    0.0    0.0   0.00   74.0    0.0    0.0  
50000  32.08   0.00    1.0  17.39    1.0    1.0  53.13  158.0    1.0    0.0  

[50001 rows x 46 columns]

In [9]:
Alpha = [chr(ord('A')+i) for i in range(26)]

In [10]:
class_mapping = {'A':1,'B':2,'C':3,'D':4,'E':5,'F':6,'G':7,'H':8,'I':9,'J':10,'K':11,'L':12,'M':13,'N':14,'O':15,'P':16,'Q':17,'R':18,'S':19,'T':20,'U':21,'V':22,'W':23,'X':24,'Y':25,'Z':26}
names = ['COL3', 'COL4','COL5','COL19']
# onehot

for name in names:
    if name in data.columns:
        temp_pd = pd.get_dummies(list(data[name])+Alpha,dtype=int)[:-26]
        temp_pd.columns = [i + name[-1] for i in list(temp_pd.columns)]
        data = pd.concat([data.drop(columns=[name]), temp_pd],axis=1)
        
        temp_pd = pd.get_dummies(list(quiz[name])+Alpha,sparse=True,dtype=int)[:-26]
        temp_pd.columns = [i + name[-1] for i in list(temp_pd.columns)]
        quiz = pd.concat([quiz.drop(columns=[name]), temp_pd],axis=1)        
# print(pd.get_dummies(list(data[names[3]])))
# print(enc.fit_transform(data[names[0]]))
# for name in names:
#     if name in data.columns:
#         data[name] = data[name].map(class_mapping)
#         quiz[name] = quiz[name].map(class_mapping)

for name in data.columns[1:]:
        data[name] = data[name].fillna(data[name].mean())
for name in quiz.columns[1:]:
        quiz[name] = quiz[name].fillna(quiz[name].mean())
data

ID  CHANNEL_A  CHANNEL_B  CHANNEL_C        COL1  COL2   COL6  \
0      T000689          0          0          0   28.102886    58   0.00   
1      T000848          0          1          0    6.000000    49   0.00   
2      T000967          0          0          0   28.102886    55   0.00   
3      T001249          0          0          0   17.000000    36   0.00   
4      T001524          0          1          0   22.000000    43   0.00   
...        ...        ...        ...        ...         ...   ...    ...   
99995  T034492          0          1          0   28.102886    30   0.00   
99996  T034682          0          0          0    4.000000    49   0.00   
99997  T035207          0          0          0   18.000000    30  60.78   
99998  T035311          0          0          0  347.000000    33   9.74   
99999  T037255          0          1          0   28.102886    34  58.43   

       COL7  COL8        COL9  ...  Q9  R9  S9  T9  U9  V9  W9  X9  Y9  Z9  
0         0     1   65.718368  ...   0   0   0   0   0   0   0   0   0   0  
1         0     1   24.000000  ...   0   0   0   0   0   0   0   0   0   0  
2         0     1   65.718368  ...   0   0   0   0   0   0   0   0   0   0  
3         0     0   41.000000  ...   0   0   0   0   0   0   0   0   0   0  
4         1     0   52.000000  ...   0   0   0   0   0   0   0   0   0   0  
...     ...   ...         ...  ...  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  
99995     0     0   65.718368  ...   0   0   0   0   0   0   0   0   0   0  
99996     0     0   29.000000  ...   0   0   0   0   0   0   0   0   0   0  
99997     0     0   40.000000  ...   0   0   0   0   0   0   0   0   0   0  
99998     2     1  377.000000  ...   0   0   0   0   0   0   0   0   0   0  
99999     2     0   65.718368  ...   0   0   0   0   0   0   0   0   0   0  

[100000 rows x 149 columns]

In [11]:
quiz

ID        COL1  COL2   COL6  COL7  COL8        COL9       COL10  \
0      T500001   26.000000  54.0   0.00   0.0   1.0   63.000000  101.000000   
1      T500002   22.000000  55.0   0.92   0.0   1.0   43.000000   72.000000   
2      T500003   50.336102  55.0   0.00   0.0   0.0   79.467306  110.950426   
3      T500004  106.000000  47.0  22.46   0.0   1.0  185.000000  206.000000   
4      T500005   18.000000  55.0  22.26   2.0   0.0   43.000000   67.000000   
...        ...         ...   ...    ...   ...   ...         ...         ...   
49996  T549996   39.000000  30.0  55.75   0.0   1.0   72.000000   95.000000   
49997  T549997   15.000000  37.0   0.00   0.0   0.0   38.000000   72.000000   
49998  T549998   46.000000  29.0   0.46   0.0   0.0   65.000000   89.000000   
49999  T549999   61.000000  35.0   0.00   0.0   1.0   98.000000  129.000000   
50000  T550000   36.000000  41.0   0.00   0.0   0.0   58.000000   78.000000   

       COL11  COL12  ...  Q9  R9  S9  T9  U9  V9  W9  X9  Y9  Z9  
0        0.0    0.0  ...   0   0   0   0   0   0   0   0   0   0  
1        0.0    0.0  ...   0   0   0   0   0   0   0   0   0   0  
2        0.0    0.0  ...   0   0   0   0   0   0   0   0   0   0  
3        0.0    0.0  ...   0   0   0   0   0   0   0   0   0   0  
4        0.0    0.0  ...   0   0   0   0   0   0   0   0   0   0  
...      ...    ...  ...  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  
49996    0.0    0.0  ...   0   0   0   0   0   0   0   0   0   0  
49997    0.0    0.0  ...   0   0   0   0   0   0   0   0   0   0  
49998    0.0    0.0  ...   0   0   0   0   0   0   0   0   0   0  
49999    0.0    0.0  ...   0   0   0   0   0   0   0   0   0   0  
50000    0.0    0.0  ...   0   0   0   0   0   0   0   0   0   0  

[50001 rows x 146 columns]

In [12]:
quiz.sample(n=len(data),replace=True)

ID       COL1  COL2   COL6  COL7  COL8        COL9       COL10  \
27141  T527141  46.000000  29.0  37.15   0.0   0.0   65.000000   87.000000   
16673  T516673  62.000000  41.0   0.00   0.0   1.0   80.000000  102.000000   
32213  T532213  50.336102  37.0   0.46   4.0   1.0   79.467306  110.950426   
43000  T543000  62.000000  38.0   0.00   0.0   1.0   84.000000  108.000000   
18995  T518995   9.000000  39.0   0.00   0.0   0.0   43.000000   60.000000   
...        ...        ...   ...    ...   ...   ...         ...         ...   
22890  T522890  43.000000  33.0   1.45   0.0   1.0   67.000000   97.000000   
40065  T540065  38.000000  32.0  12.90   0.0   1.0   72.000000  176.000000   
48977  T548977  87.000000  45.0   0.00   0.0   1.0  224.000000  110.950426   
5018   T505019  45.000000  42.0  37.07   0.0   0.0   65.000000  206.000000   
49718  T549718  62.000000  23.0   0.00   0.0   0.0  114.000000  141.000000   

       COL11  COL12  ...  Q9  R9  S9  T9  U9  V9  W9  X9  Y9  Z9  
27141    0.0    0.0  ...   0   0   0   0   0   0   0   0   0   0  
16673    0.0    0.0  ...   0   0   0   0   0   0   0   0   0   0  
32213    0.0    0.0  ...   0   0   0   0   0   0   0   0   0   0  
43000    0.0    0.0  ...   0   0   0   0   0   0   0   0   0   0  
18995    0.0    0.0  ...   0   0   0   0   0   0   0   0   0   0  
...      ...    ...  ...  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  
22890    0.0    0.0  ...   0   0   0   0   0   0   0   0   0   0  
40065    0.0    0.0  ...   0   0   0   0   0   0   0   0   0   0  
48977    0.0    0.0  ...   0   0   0   0   0   0   0   0   0   0  
5018     0.0    0.0  ...   0   0   0   0   0   0   0   0   0   0  
49718    0.0    0.0  ...   0   0   0   0   0   0   0   0   0   0  

[100000 rows x 146 columns]

In [13]:
def get_adv_feats(df_train, df_test, feats):
    df_train['adv'] = 1
    df_test['adv'] = 0
    df = pd.concat([df_train, df_test]).reset_index(drop=True)
    
    params = {
        'learning_rate': 0.1,
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'seed': 2222,
        'n_jobs': 4,
        'verbose': -1,
    }
    
    fold_num = 5
    seeds = [2222]
    new_feats = []
    for f in feats:
        oof = np.zeros(len(df))
        for seed in seeds:
            kf = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=seed)
            for fold, (train_idx, val_idx) in enumerate(kf.split(df[[f]], df['adv'])):
                train = lgb.Dataset(df.loc[train_idx, [f]],
                                    df.loc[train_idx, 'adv'])
                val = lgb.Dataset(df.loc[val_idx, [f]],
                                  df.loc[val_idx, 'adv'])
                model = lgb.train(params, train, valid_sets=[val], num_boost_round=10000,  # feval=recall_score,
                                  callbacks=[lgb.early_stopping(100), lgb.log_evaluation(-1)])
                oof[val_idx] += model.predict(df.loc[val_idx, [f]]) / len(seeds)
                score = auc(df.loc[val_idx, 'adv'], oof[val_idx])
                print(f,' --- ', score)
                if score > 0.8:
                    print('--------------------------------------', f, score)
                else:
                    new_feats.append(f)
                break
    return new_feats

# print(data.drop(columns=['CHANNEL_A','CHANNEL_B','CHANNEL_C']).columns == quiz.columns)
feats = quiz.drop(columns=['ID']).columns
feats = get_adv_feats(data.drop(columns=['ID','CHANNEL_A','CHANNEL_B','CHANNEL_C']),
                      quiz.drop(columns=['ID']).sample(n=len(data),replace=True), feats)


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[326]	valid_0's auc: 0.932929
COL1  ---  0.5246113527926826
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	valid_0's auc: 0.610368
COL2  ---  0.49861091266470214
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[100]	valid_0's auc: 0.520723
COL6  ---  0.5075808238262185
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	valid_0's auc: 0.51764
COL7  ---  0.5038543933910365
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	valid_0's auc: 0.51045
COL8  ---  0.5001561404667298
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[211]	valid_0's auc: 0.908464
COL9  ---  0.6089589215699456
Training until validation scores don't improve for 100 rounds
Ear

Early stopping, best iteration is:
[1]	valid_0's auc: 0.5
T5  ---  0.5
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	valid_0's auc: 0.5
U5  ---  0.5
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	valid_0's auc: 0.5
V5  ---  0.5
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	valid_0's auc: 0.5
W5  ---  0.5
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	valid_0's auc: 0.5
X5  ---  0.5
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	valid_0's auc: 0.5
Y5  ---  0.5
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	valid_0's auc: 0.5
Z5  ---  0.5
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	valid_0's auc: 0.5
A9  ---  0.

In [14]:
feats = ['ID','CHANNEL_A','CHANNEL_B','CHANNEL_C'] + feats
print(feats)
for col in data.columns:
    if col in feats:
        continue
    data = data.drop(columns=[col])
for col in quiz.columns:
    if col in feats:
        continue
    quiz = quiz.drop(columns=[col])

['ID', 'CHANNEL_A', 'CHANNEL_B', 'CHANNEL_C', 'COL1', 'COL2', 'COL6', 'COL7', 'COL8', 'COL9', 'COL10', 'COL11', 'COL12', 'COL15', 'COL16', 'COL17', 'COL18', 'COL20', 'COL22', 'COL23', 'COL24', 'COL26', 'COL27', 'COL28', 'COL29', 'COL30', 'COL32', 'COL33', 'COL34', 'COL35', 'COL36', 'COL38', 'COL39', 'COL40', 'COL41', 'COL42', 'COL43', 'COL44', 'COL47', 'COL48', 'COL49', 'COL50', 'COL51', 'COL53', 'COL54', 'A3', 'B3', 'C3', 'D3', 'E3', 'F3', 'G3', 'H3', 'I3', 'J3', 'K3', 'L3', 'M3', 'N3', 'O3', 'P3', 'Q3', 'R3', 'S3', 'T3', 'U3', 'V3', 'W3', 'X3', 'Y3', 'Z3', 'A4', 'B4', 'C4', 'D4', 'E4', 'F4', 'G4', 'H4', 'I4', 'J4', 'K4', 'L4', 'M4', 'N4', 'O4', 'P4', 'Q4', 'R4', 'S4', 'T4', 'U4', 'V4', 'W4', 'X4', 'Y4', 'Z4', 'A5', 'B5', 'C5', 'D5', 'E5', 'F5', 'G5', 'H5', 'I5', 'J5', 'K5', 'L5', 'M5', 'N5', 'O5', 'P5', 'Q5', 'R5', 'S5', 'T5', 'U5', 'V5', 'W5', 'X5', 'Y5', 'Z5', 'A9', 'B9', 'C9', 'D9', 'E9', 'F9', 'G9', 'H9', 'I9', 'J9', 'K9', 'L9', 'M9', 'N9', 'O9', 'P9', 'Q9', 'R9', 'S9', 'T9', 'U9

In [15]:
cnt = 0
for i in data['CHANNEL_A']:
    if i == 1:
        cnt += 1
print(cnt/len(data))


0.05497


In [16]:
# for i in data.columns[4:]:
#     cnt = 0
#     for j in range(0, len(data[i])):
#         if np.isnan(data.loc[j,i]):
#             data.loc[j,i] = min(data[i].min() - 1,0)
#             data.loc[j,i] = data[i].mean()
# data.head(20)

In [17]:
from sklearn.model_selection import train_test_split
dat_train, dat_test = train_test_split(data, random_state=0, test_size=0.2, shuffle=True)
# A, B, C = data['CHANNEL_A'], data['CHANNEL_B'], data['CHANNEL_C']
dat_train.drop(columns=['ID','CHANNEL_A','CHANNEL_B','CHANNEL_C'])

COL1  COL2   COL6  COL7  COL8       COL9      COL10  COL11  COL12  \
10382  17.000000    31  54.34     0     0  35.000000   62.00000      0      0   
73171  31.000000    38   0.00     1     0  63.000000  125.00000      0      0   
30938  18.000000    26  54.61     0     0  49.000000  125.00000      0      0   
99310  28.102886    33   0.00     0     1  65.718368  108.48216      0      0   
58959   3.000000    39   0.00     0     1  21.000000   41.00000      0      0   
...          ...   ...    ...   ...   ...        ...        ...    ...    ...   
21243  24.000000    32  39.72     0     1  52.000000   88.00000      0      2   
45891  12.000000    39   0.00     1     1  92.000000  121.00000      0      0   
42613  28.102886    33   0.00     0     1  65.718368  108.48216      0      0   
43567   8.000000    40   0.00     0     0  29.000000   53.00000      0      0   
68268  28.102886    61   0.00     0     0  65.718368  108.48216      0      0   

       COL15  ...  Q9  R9  S9  T9  U9  V9  W9  X9  Y9  Z9  
10382  59.99  ...   0   0   0   0   0   0   0   0   0   0  
73171  63.03  ...   0   0   0   0   0   0   0   0   0   0  
30938  63.49  ...   0   0   0   0   0   0   0   0   0   0  
99310  68.07  ...   0   0   0   0   0   0   0   0   0   0  
58959  69.38  ...   0   0   0   0   0   0   0   0   0   0  
...      ...  ...  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  
21243  56.10  ...   0   0   0   0   0   0   0   0   0   0  
45891   4.22  ...   0   0   0   0   0   0   0   0   0   0  
42613  64.67  ...   0   0   0   0   0   0   0   0   0   0  
43567  65.85  ...   0   0   0   0   0   0   0   0   0   0  
68268  74.01  ...   0   0   0   0   0   0   0   0   0   0  

[80000 rows x 145 columns]

In [18]:
IDs = [[i] for i in quiz['ID']]

In [19]:
quiz

ID        COL1  COL2   COL6  COL7  COL8        COL9       COL10  \
0      T500001   26.000000  54.0   0.00   0.0   1.0   63.000000  101.000000   
1      T500002   22.000000  55.0   0.92   0.0   1.0   43.000000   72.000000   
2      T500003   50.336102  55.0   0.00   0.0   0.0   79.467306  110.950426   
3      T500004  106.000000  47.0  22.46   0.0   1.0  185.000000  206.000000   
4      T500005   18.000000  55.0  22.26   2.0   0.0   43.000000   67.000000   
...        ...         ...   ...    ...   ...   ...         ...         ...   
49996  T549996   39.000000  30.0  55.75   0.0   1.0   72.000000   95.000000   
49997  T549997   15.000000  37.0   0.00   0.0   0.0   38.000000   72.000000   
49998  T549998   46.000000  29.0   0.46   0.0   0.0   65.000000   89.000000   
49999  T549999   61.000000  35.0   0.00   0.0   1.0   98.000000  129.000000   
50000  T550000   36.000000  41.0   0.00   0.0   0.0   58.000000   78.000000   

       COL11  COL12  ...  Q9  R9  S9  T9  U9  V9  W9  X9  Y9  Z9  
0        0.0    0.0  ...   0   0   0   0   0   0   0   0   0   0  
1        0.0    0.0  ...   0   0   0   0   0   0   0   0   0   0  
2        0.0    0.0  ...   0   0   0   0   0   0   0   0   0   0  
3        0.0    0.0  ...   0   0   0   0   0   0   0   0   0   0  
4        0.0    0.0  ...   0   0   0   0   0   0   0   0   0   0  
...      ...    ...  ...  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  
49996    0.0    0.0  ...   0   0   0   0   0   0   0   0   0   0  
49997    0.0    0.0  ...   0   0   0   0   0   0   0   0   0   0  
49998    0.0    0.0  ...   0   0   0   0   0   0   0   0   0   0  
49999    0.0    0.0  ...   0   0   0   0   0   0   0   0   0   0  
50000    0.0    0.0  ...   0   0   0   0   0   0   0   0   0   0  

[50001 rows x 146 columns]

In [20]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
# scaler = StandardScaler()
scaler = MinMaxScaler()

train_x = pd.DataFrame(scaler.fit_transform(dat_train.drop(columns=['ID','CHANNEL_A','CHANNEL_B','CHANNEL_C'])),
                       columns=dat_train.columns[4:])
print(train_x)
train_y = (dat_train[['CHANNEL_A','CHANNEL_B','CHANNEL_C']].reset_index().drop(columns=['index']))
print(train_y)

test_x = pd.DataFrame(scaler.transform(dat_test.drop(columns=['ID','CHANNEL_A','CHANNEL_B','CHANNEL_C'])),
                       columns=dat_test.columns[4:])
print(test_x)
test_y = (dat_test[['CHANNEL_A','CHANNEL_B','CHANNEL_C']].reset_index().drop(columns=['index']))
print(test_y)

quiz = pd.DataFrame(scaler.transform(quiz.drop(columns=['ID'])),
                       columns=dat_train.columns[4:])

C:\Users\Kevin_Chen\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:869: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Users\Kevin_Chen\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:869: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


           COL1      COL2      COL6      COL7  COL8      COL9     COL10  \
0      0.042553  0.319588  0.453362  0.000000   0.0  0.088000  0.157333   
1      0.079787  0.391753  0.000000  0.111111   0.0  0.162667  0.325333   
2      0.045213  0.268041  0.455615  0.000000   0.0  0.125333  0.325333   
3      0.072082  0.340206  0.000000  0.000000   1.0  0.169916  0.281286   
4      0.005319  0.402062  0.000000  0.000000   1.0  0.050667  0.101333   
...         ...       ...       ...       ...   ...       ...       ...   
79995  0.061170  0.329897  0.331387  0.000000   1.0  0.133333  0.226667   
79996  0.029255  0.402062  0.000000  0.111111   1.0  0.240000  0.314667   
79997  0.072082  0.340206  0.000000  0.000000   1.0  0.169916  0.281286   
79998  0.018617  0.412371  0.000000  0.000000   0.0  0.072000  0.133333   
79999  0.072082  0.628866  0.000000  0.000000   0.0  0.169916  0.281286   

       COL11     COL12     COL15  ...   Q9   R9   S9   T9   U9   V9   W9   X9  \
0        0.0  0.00

C:\Users\Kevin_Chen\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:869: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Users\Kevin_Chen\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:869: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


In [21]:
F1_avg = 0
best_score = 0

ans = IDs

eval_metrics = ['error','auc']
max_depths = [5,6,7,8,9]
lambdas = [2, 0, 1, 4]
gammas = [0, 0.1, 0.5, 0.75, 1]
subsamples = [0.75, 0.65, 0.85]
colsample_bytrees = [0.75, 0.6, 0.7, 0.8]
min_child_weights = [1, 2, 4]
etas = [0.015, 0.05]
reg_alphas = [0, 0.01,0.05, 0.1, 1]

F1_Sores = {'CHANNEL_A':0,'CHANNEL_B':0,'CHANNEL_C':0}


best_params={'booster':'gbtree',
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        'max_depth':6,
        'lambda':2,
        'gamma':0,
        'subsample':0.75,
        'colsample_bytree':0.75,
        'min_child_weight':4,
        'eta': 0.005,
        'nthread':-1}
Params = {'CHANNEL_A':best_params,'CHANNEL_B':best_params,'CHANNEL_C':best_params}
# 'scale_pos_weight':pos_weight

In [22]:
Attributes = ['CHANNEL_A','CHANNEL_B','CHANNEL_C']
for i in Attributes[:]:
    class_weights = {0: len(train_y[i])-train_y[i].sum(), 1: train_y[i].sum()}
    pos_weight = class_weights[0] / class_weights[1]
    print(pos_weight)

17.210789893011608
2.892375808884348
110.42061281337047


In [23]:
def XGB_train(train_x,train_y,test_x,test_y,quiz,step,params):
    
    dtrain = xgb.DMatrix(train_x, label=train_y)
    dtest = xgb.DMatrix(test_x)
    dquiz = xgb.DMatrix(quiz)
    class_weights = {0: len(train_y)-train_y.sum(), 1: train_y.sum()}
    pos_weight = class_weights[0] / class_weights[1]
    params['scale_pos_weight'] = pos_weight
#     params={'booster':'gbtree',
#             'objective': 'binary:logistic',
#             'eval_metric': 'auc',
#             'max_depth':6,
#             'lambda':2,
#             'gamma':0,
#             'subsample':0.75,
#             'colsample_bytree':0.75,
#             'min_child_weight':4,
#             'eta': 0.005,
#             'seed':0,
#             'nthread':-1,
# #             'silent':0,
#             'scale_pos_weight':pos_weight}
    watchlist = [(dtrain,'train')]
    
    
    
    model = xgb.train(params,dtrain,num_boost_round=200,evals=watchlist,obj=None,feval=None,maximize=False,early_stopping_rounds=100,
    evals_result=None,verbose_eval=False,xgb_model=None)

    ypred = model.predict( dtest )
    y_pred = (ypred >= 0.5) * 1
#     print(step)
#     print(y_pred.sum())
#     print(len(y_pred))
#     print('AUC: %.4f' % metrics.roc_auc_score(test_y, ypred))
#     print('ACC: %.4f' % metrics.accuracy_score(test_y, y_pred))
#     print('Precesion: %.4f' % metrics.precision_score(test_y, y_pred))
#     print('Recall: %.4f' % metrics.recall_score(test_y, y_pred))
#     print('F1-score: %.4f' % metrics.f1_score(test_y, y_pred))
#     print(metrics.confusion_matrix(test_y, y_pred))
#     print()
#     plot_importance(model)
    reply = (model.predict( dquiz ) >= 0.5) * 1
    for i in range(0,len(reply)):
        ans[i].append(reply[i])
    return metrics.f1_score(test_y, y_pred)

In [24]:
def Train(param):
    ans = IDs
    F1_avg = 0
    Attributes = ['CHANNEL_A','CHANNEL_B','CHANNEL_C']
    for i in Attributes[:]:
        F1 = XGB_train(train_x,train_y[i],test_x,test_y[i],quiz,i, param)
        if F1_Sores[i] < F1:
            Params[i] = param
            F1_Sores[i] = F1
        F1_avg += F1
#     print(F1_avg/3.0)
    return F1_avg/3.0

In [ ]:
best_score = 0
for reg_alpha in reg_alphas:
    for eta in etas:
        for lambd in lambdas:
            for gamma in gammas:
                for subsample in subsamples:
                    for colsample_bytree in colsample_bytrees:
                        for min_child_weight in min_child_weights:
                            for max_depth in max_depths:
                                for eval_metric in eval_metrics:
                                    temp_params={'booster':'gbtree',
                                                'objective': 'binary:logistic',
                                                'eval_metric': eval_metric,
                                                'max_depth':max_depth,
                                                'lambda':lambd,
                                                'gamma':gamma,
                                                'subsample':subsample,
                                                'colsample_bytree':colsample_bytree,
                                                'min_child_weight':min_child_weight,
                                                'eta': eta,
                                                'nthread':-1}
                                    f1_s = Train(temp_params)
                                    if f1_s > best_score:
                                        best_params = temp_params
                                        best_score = f1_s
                                        print(best_score)
                                        print(best_params)
                                        print("==========================")

0.22680064048101986
{'booster': 'gbtree', 'objective': 'binary:logistic', 'eval_metric': 'error', 'max_depth': 5, 'lambda': 2, 'gamma': 0, 'subsample': 0.75, 'colsample_bytree': 0.75, 'min_child_weight': 1, 'eta': 0.015, 'nthread': -1, 'scale_pos_weight': 110.42061281337047}
0.23056551868743155
{'booster': 'gbtree', 'objective': 'binary:logistic', 'eval_metric': 'error', 'max_depth': 6, 'lambda': 2, 'gamma': 0, 'subsample': 0.75, 'colsample_bytree': 0.75, 'min_child_weight': 1, 'eta': 0.015, 'nthread': -1, 'scale_pos_weight': 110.42061281337047}
0.23245882188502684
{'booster': 'gbtree', 'objective': 'binary:logistic', 'eval_metric': 'error', 'max_depth': 9, 'lambda': 2, 'gamma': 0, 'subsample': 0.75, 'colsample_bytree': 0.75, 'min_child_weight': 1, 'eta': 0.015, 'nthread': -1, 'scale_pos_weight': 110.42061281337047}
0.23344220762504633
{'booster': 'gbtree', 'objective': 'binary:logistic', 'eval_metric': 'error', 'max_depth': 9, 'lambda': 2, 'gamma': 0, 'subsample': 0.75, 'colsample_byt

In [ ]:
best_score = max(best_score,F1_avg/3.0)
best_score

In [ ]:
Params

In [ ]:
F1_Sores

In [ ]:
sum(list(F1_Sores.values()))/3.0

In [ ]:
best_score

0.2319161220271427

In [ ]:
ans

In [ ]:
Output = pd.DataFrame(ans,columns=['ID','CHANNEL_A','CHANNEL_B','CHANNEL_C'],index=None)
Output

In [ ]:
Output.to_csv("output.csv",index=False)

{'CHANNEL_A': {'booster': 'gbtree',
  'objective': 'binary:logistic',
  'eval_metric': 'error',
  'max_depth': 8,
  'lambda': 2,
  'gamma': 0.1,
  'subsample': 0.85,
  'colsample_bytree': 0.75,
  'min_child_weight': 1,
  'eta': 0.015,
  'nthread': -1,
  'scale_pos_weight': 110.42061281337047},
 'CHANNEL_B': {'booster': 'gbtree',
  'objective': 'binary:logistic',
  'eval_metric': 'error',
  'max_depth': 8,
  'lambda': 2,
  'gamma': 1,
  'subsample': 0.85,
  'colsample_bytree': 0.75,
  'min_child_weight': 4,
  'eta': 0.015,
  'nthread': -1,
  'scale_pos_weight': 110.42061281337047},
 'CHANNEL_C': {'booster': 'gbtree',
  'objective': 'binary:logistic',
  'eval_metric': 'error',
  'max_depth': 9,
  'lambda': 2,
  'gamma': 0.75,
  'subsample': 0.65,
  'colsample_bytree': 0.8,
  'min_child_weight': 4,
  'eta': 0.015,
  'nthread': -1,
  'scale_pos_weight': 110.42061281337047}}